In [1]:
import clickhouse_connect
from embedding import E5LargeEmbeddingFunction
from similarity import bm25_ensemble_with_crossenc_answer, prep_query, bm25_fewshot_with_cross_encoder
from pprint import pprint
import requests
import json
import pandas as pd

/home/veidlink/HAKATONS/cb-purple-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pip freeze | findstr /v "@" > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
few_shot_data = pd.read_csv('data/finetune_df_final.csv', index_col=0)
few_shot_data

,doc_id,generated_question,generated_answer,is_markdown,context_batch
0,572.0,Какое полное и сокращенное фирменное наименова...,Полное фирменное наименование: Банк развития и...,0.0,Банковский сектор Информация о кредитных ор...
1,572.0,Какие данные представлены в отчете о финансовы...,В отчете о финансовых результатах формы 102 за...,0.0,на 1 февраля на 1 марта ...
2,572.0,Какие документы содержат информацию об обязате...,Информация об обязательных нормативах и других...,0.0,134 2015 год 2014 год 2013...
3,572.0,Какие документы включены в годовую (промежуточ...,В годовую (промежуточную) отчётность включаютс...,0.0,октября на 1 ноября ...
4,312.0,Какие основные направления развития финансовых...,В тексте описаны основные направления развития...,0.0,ОснОвные направления \nразвития финансОвых \...
...,...,...,...,...,...
348,1629.0,Какова доля плохих кредитов в процентах и как ...,"Доля плохих кредитов составляет 82.8%, а соотн...",0.0,.1\n6\n\n0\n6\n\n.1\n6\n\n0\n9\n\n.1\n6\n\n1\n...
349,1629.0,Какие данные представлены на графиках по росси...,На графиках представлены данные о торговых объ...,0.0,.1\n7\n\n0\n3\n\n.1\n8\n\n0\n6\n\n.1\n8\n\n0\n...
350,1629.0,Каковы торговые объемы на Московской бирже в т...,Торговые объемы на Московской бирже составляют...,0.0,8\n\n6\n.1\n\n8\n\n9\n.1\n\n8\n\nTrading volum...
351,1629.0,Что представляют собой данные '8% 11% 11% 12%'...,Эти данные представляют собой процентные ставк...,0.0,联邦公债券，万亿卢布 企业债券，万亿卢布\n\n8% 11% 11% 12%\n2% 4% ...


**Inference к модели, захосченной через ngrok & LM Studio**

In [3]:
client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

In [62]:
system_prompt = """
### ROLE ###
Ты - дружелюбный ассистент Центрального банка Российской Федерации. 
Исчерпывающе отвечай клиентам на возникшие вопросы. Если из контекста 
невозможно дать ответ на вопрос, пиши «Я не могу ответить на этот вопрос»."""

In [63]:
def message_llm(system_prompt, user_input, temperature=0.6):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
          # "max_tokens": -1,
          # "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")

### Zero-shot prompting

In [80]:
rag_prompt = """
### CONTEXT ###

1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###

На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

question = 'Какую информационную политику ЦБ проводит в регионах? Как ЦБ представлен в регионах?'
query = prep_query(question)

result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

type_of_source1, type_of_source2, type_of_source3  = '', '', ''
link_to_source1, link_to_source2, link_to_source3 = '', '', ''
batch_1, batch_2, batch_3 = '', '', ''

for idx, batch in enumerate(start=1, iterable=result):
    if idx==1:
        if batch[5]:
            type_of_source1 = 'ТАБЛИЦА'
        else:
            type_of_source1 = 'ТЕКСТ'
        link_to_source1 = batch[2]
        batch_1 = batch[3]

    if idx==2:
        if batch[5]:
            type_of_source2 = 'ТАБЛИЦА'
        else:
            type_of_source2 = 'ТЕКСТ'
        link_to_source2 = batch[2]
        batch_2 = batch[3]

    if idx==3:
        if batch[5]:
            type_of_source3 = 'ТАБЛИЦА'
        else:
            type_of_source3 = 'ТЕКСТ'
        link_to_source3 = batch[2]
        batch_3 = batch[3]

formatted_rag_prompt = rag_prompt.format(
    type_of_source1=type_of_source1,
    link_to_source1=link_to_source1,
    batch_1=batch_1,
    type_of_source2=type_of_source2,
    link_to_source2=link_to_source2,
    batch_2=batch_2,
    type_of_source3=type_of_source3,
    link_to_source3=link_to_source3,
    batch_3=batch_3,
    QUESTION=question
)

answer = message_llm(system_prompt, formatted_rag_prompt)

print(f"\n{answer}")
print(f'\nИспользованные источники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}')


Центральный банк Российской Федерации (ЦБ) действует на всей территории Российской Федерации и осуществляет свою деятельность в соответствии с законодательством. В регионах ЦБ представлен через свои структуры, такие как отделения ЦБ, которые обеспечивают выполнение банковских операций, а также управление денежной эмиссией и контроль над банками.

В рамках информационной политики ЦБ проводит различные мероприятия для обеспечения общественного доверия к денежно-кредитной системе, в том числе публикация информации о своих действиях и политике. Также ЦБ использует различные каналы коммуникации, такие как сайт ЦБ (cbr.ru), пресс-службу и другие способы, для достоверной и транспарентной информации о своей деятельности и политике в регионах.

Использованные источники: 
https://cbr.ru//Collection/Collection/File/8485/obs_1702.pdf, 
https://cbr.ru//Collection/Collection/File/8476/obs_181.pdf, 
https://cbr.ru//Collection/Collection/File/8482/obs_1705.pdf


### Few-shot-promoting

In [64]:
rag_prompt_fw_ex = """
### EXAMPLES ###

1) Пример 1:
КОНТЕКСТ: {context_1}
ВОПРОС: {question_1}
ОТВЕТ: {answer_1}

2) Пример 2:
КОНТЕКСТ: {context_2}
ВОПРОС: {question_2}
ОТВЕТ: {answer_2}

3) Пример 3:
КОНТЕКСТ: {context_3}
ВОПРОС: {question_3}
ОТВЕТ: {answer_3}

### CONTEXT ###

1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###

На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

In [65]:
question = 'Что означает "политика единого голоса" в ЦБ?'
query = prep_query(question)

result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

type_of_source1, type_of_source2, type_of_source3  = '', '', ''
link_to_source1, link_to_source2, link_to_source3 = '', '', ''
batch_1, batch_2, batch_3 = '', '', ''

for idx, batch in enumerate(start=1, iterable=result):
    if idx==1:
        if batch[5]:
            type_of_source1 = 'ТАБЛИЦА'
        else:
            type_of_source1 = 'ТЕКСТ'
        link_to_source1 = batch[2]
        batch_1 = batch[3]

    if idx==2:
        if batch[5]:
            type_of_source2 = 'ТАБЛИЦА'
        else:
            type_of_source2 = 'ТЕКСТ'
        link_to_source2 = batch[2]
        batch_2 = batch[3]

    if idx==3:
        if batch[5]:
            type_of_source3 = 'ТАБЛИЦА'
        else:
            type_of_source3 = 'ТЕКСТ'
        link_to_source3 = batch[2]
        batch_3 = batch[3]

few_shot_examples = bm25_fewshot_with_cross_encoder(bm25_n_results=100, cr_enc_n_results=3, df=few_shot_data, query=query)
formatted_examples = '\n\n'.join([f"Пример {i+1}:\nКОНТЕКСТ: {ex['context']}\nВОПРОС: {ex['question']}\nОТВЕТ: {ex['answer']}" for i, ex in enumerate(few_shot_examples)])

formatted_rag_prompt = rag_prompt_fw_ex.format(
    context_1=few_shot_examples[0]['context'],
    question_1=few_shot_examples[0]['question'],
    answer_1=few_shot_examples[0]['answer'],
    context_2=few_shot_examples[1]['context'],
    question_2=few_shot_examples[1]['question'], 
    answer_2=few_shot_examples[1]['answer'],
    context_3=few_shot_examples[2]['context'],
    question_3=few_shot_examples[2]['question'],  
    answer_3=few_shot_examples[2]['answer'],  
    type_of_source1=type_of_source1,
    link_to_source1=link_to_source1,
    batch_1=batch_1,
    type_of_source2=type_of_source2,
    link_to_source2=link_to_source2,
    batch_2=batch_2,
    type_of_source3=type_of_source3,
    link_to_source3=link_to_source3,
    batch_3=batch_3,
    QUESTION=question
)

answer = message_llm(system_prompt, formatted_rag_prompt)

print(f"\n{answer}")
print(f'\nИспользованные источники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}')


Ответ: Политика единого голоса - это правило, согласно которому одному члену Совета Федерации может принадлежать не более одного депутатского мандата. Это означает, что человек, избранный в Государственную Думу, не может одновременно занимать пост сенатора.

Использованные источники: 
https://cbr.ru//Content/Document/File/124614/api_kbki_1_1_20210720.PDF, 
https://cbr.ru//Content/Document/File/124614/api_kbki_1_1_20210720.PDF, 
https://cbr.ru//Content/Document/File/124614/api_kbki_1_1_20210720.PDF


### Self-consistency promoting

In [72]:
rag_prompt = """
### CONTEXT ###

1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###

На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

sc_prompt = """
### CONTEXT ###

1) МНЕНИЕ 1: 
- {answer1}

2) МНЕНИЕ 2: 
- {answer2}

3) МНЕНИЕ 3:
- {answer3}

### INSTRUCTION ###
На основании приведенного контекста, дай итоговый ответ клиенту на его вопрос по принципу большинства.
"""

In [79]:
question = 'Какую информационную политику ЦБ проводит в регионах? Как ЦБ представлен в регионах?'
query = prep_query(question)

result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

type_of_source1, type_of_source2, type_of_source3  = '', '', ''
link_to_source1, link_to_source2, link_to_source3 = '', '', ''
batch_1, batch_2, batch_3 = '', '', ''


for idx, batch in enumerate(start=1, iterable=result):
    if idx==1:
        if batch[5]:
            type_of_source1 = 'ТАБЛИЦА'
        else:
            type_of_source1 = 'ТЕКСТ'
        link_to_source1 = batch[2]
        batch_1 = batch[3]

    if idx==2:
        if batch[5]:
            type_of_source2 = 'ТАБЛИЦА'
        else:
            type_of_source2 = 'ТЕКСТ'
        link_to_source2 = batch[2]
        batch_2 = batch[3]

    if idx==3:
        if batch[5]:
            type_of_source3 = 'ТАБЛИЦА'
        else:
            type_of_source3 = 'ТЕКСТ'
        link_to_source3 = batch[2]
        batch_3 = batch[3]

formatted_rag_prompt = rag_prompt.format(
    type_of_source1=type_of_source1,
    link_to_source1=link_to_source1,
    batch_1=batch_1,
    type_of_source2=type_of_source2,
    link_to_source2=link_to_source2,
    batch_2=batch_2,
    type_of_source3=type_of_source3,
    link_to_source3=link_to_source3,
    batch_3=batch_3,
    QUESTION=question
)

answer1 = message_llm(system_prompt, formatted_rag_prompt)
answer2 = message_llm(system_prompt, formatted_rag_prompt)
answer3 = message_llm(system_prompt, formatted_rag_prompt)

formatted_som_prompt = sc_prompt.format(answer1=answer1,answer2=answer2,answer3=answer3)
final_answer = message_llm(system_prompt, formatted_som_prompt)

print(f"\n{final_answer}")
print(f'\nИспользованные источники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}')



Центральный банк Российской Федерации проводит информационную политику, направленную на обеспечение открытости и транспарентности деятельности ЦБ, а также предоставление объективной, достоверной и актуальной информации о состоянии и перспективах развития финансовой системы России. В регионах ЦБ представлен через свои структурные подразделения - отделения по банковскому надзору, которые функционируют в каждом субъекте Российской Федерации. Отделения по банковскому надзору осуществляют контроль за деятельностью кредитных организаций (банков), а также принимают решения о разрешении деятельности, введении исключения из перечня кредитных организаций.

Использованные источники: 
https://cbr.ru//Collection/Collection/File/8497/obs_1602.pdf, 
https://cbr.ru//Collection/Collection/File/8482/obs_1705.pdf, 
https://cbr.ru//Collection/Collection/File/8506/obs_1505.pdf
